# Практическое занятие №5

П.Н. Советов, РТУ МИРЭА

Решать эту и дальнейшие задачи удобнее в парном составе: один студент занимается написанием кода программы и пытается внести в код такие ошибки, которые будет сложно протестировать другому студенту.

Пример программы с ошибкой:

```Python
def bucketsort(arr, k):
    counts = [0] * k
    for x in arr:
        counts[x] += 1

    sorted_arr = []
    for i, count in enumerate(arr):
        sorted_arr.extend([i] * count)

    return sorted_arr
```

Другие примеры можно взять [отсюда](https://github.com/jkoppel/QuixBugs/tree/master/python_programs).

**Задача 1**

Использование модуля doctest

1. Добавьте документацию к программе в виде docstring-строки.
1. Укажите примеры в формате doctest. Примеры должны охватывать граничные случаи.
1. Протестируйте программу с помощью вызова модуля doctest.
1. Перенесите примеры в отдельный файл и снова протестируйте программу.

**Задача 2**

Использование модуля pytest

1. Создайте отдельный файл для тестирования в который поместите тестирующие функции (не менее двух).
1. Упростите код с помощью добавления fixture-функций.
1. Добавьте параметризацию.
1. Добавьте макетный код.

**Задача 3**

Использование модуля coverage

1. Получите статистику по покрытию операторов.
1. Получите статистику по покрытию ветвей.
1. Постарайтесь изменить код исходной программы так, чтобы затруднить получение 100% покрытия.
1. Реализуйте вывод статистики о покрытии в HTML-представлении.

**Задача 4**

Прототип системы мутационного тестирования приведен ниже. Попробуйте разобраться в том, как работает этот код. Вам поможет документация к модулю ast.

Функция `mut_test` принимает на вход тестируемую функцию и функцию, осуществляющую тестирование с помощью `assert`. Функции необходимо добавлять к приведенному ниже модулю.

In [1]:
import random
from collections import defaultdict
import inspect
from ast import *


class Locator(NodeVisitor):
    def __init__(self):
        self.locs = defaultdict(list)

    def visit(self, node):
        self.locs[type(node)].append(node)
        self.generic_visit(node)


class Mutator(NodeTransformer):
    def __init__(self, locs):
        # TODO
        node_type = random.choice([...])
        self.target = random.choice(locs[node_type])

    def visit(self, node):
        if self.target != node:
            return self.generic_visit(node)
        # TODO


def mutate_code(src, max_changes):
    tree = parse(src)
    loc = Locator()
    loc.visit(tree)
    mut = Mutator(loc.locs)
    for _ in range(random.randint(1, max_changes)):
        mut.visit(tree)
    return unparse(tree)


def make_mutants(func, size, max_changes):
    mutant = src = unparse(parse(inspect.getsource(func)))
    mutants = [src]
    while len(mutants) < size + 1:
        while mutant in mutants:
            mutant = mutate_code(src, max_changes)
        mutants.append(mutant)
    return mutants[1:]


def mut_test(func, test, size=20, max_changes=1):
    survived = []
    mutants = make_mutants(func, size, max_changes)
    for mutant in mutants:
        try:
            exec(mutant, globals())
            test()
            survived.append(mutant)
        except:
            pass
    return survived


1. Сгенерируйте несколько программ-мутантов.
1. Сгенерируйте несколько программ-мутантов при 100% покрытии ветвей.
1. Постарайтесь добиться полного отсутствия мутантов.
1. Было бы удобно общий код мутационного тестирования вынести в отдельный модуль. Увы, это не сработает. Почему?

**Задача 5**

Использование модуля deal

1. Добавьте к программе контракты `pre`, `post`, `ensure`, `raises`, `reason`.
1. Найдите программу, для которой контракт `has` будет полезен.
1. Для программы с классами используйте инвариант `inv`.
1. Для тестирования контрактов используйте pytest.
1. Реализуйте контракт, выполнение которого `deal` проверяет статически. Какие ограничения имеют статически проверяемые контракты?

**Задача 6**

Использование модуля hypothesis

1. Реализуйте тестирование для численных входных данных, строк, списков.
1. Реализуйте тестирование для словарей, деревьев, графов.
1. Используйте при создании свойств как можно больше категорий, перечисленных в лекции.
1. Найдите подходящую программу и реализуйте для нее тестирование на основе модели.
1. Разберитесь, для чего в библиотеке hypothesis используются bundles и примените их в тестировании на основе моделей.

**Задача 7**

Формальная верификация головоломок из компьютерных игр

Одной из важных проблем, стоящих перед разработчиком компьютерных игр, является создание интересных головоломок, в которых отсутствуют тупиковые состояния (состояния, из которых нельзя достичь цели).

Формальную верификацию проведем следующим образом:

1. Реализовать игровую ситуацию в виде некоторого количества локаций с указанием перечня возможных действий в каждой из них.
1. Сгенерировать по реализованной игровой ситуации граф всех возможных игровых состояний, в котором ребра задают переходы из состояние в состояние.
1. Проанализировать граф состояний на предмет проверяемого игрового свойства.

Рассмотрим следующую игровую ситуацию из [PuzzleGraph](https://runevision.itch.io/puzzlegraph):

![](images/puzzlegraph.png)

Здесь Goal означает целевое состояние, а состояние с двумя зелеными кругами обозначает старт. Справа от стартового состояния используется специальное ребро, по которому позволяется двигаться только в одну сторону, вправо.

Ниже приведено описание рассмотренной игровой ситуации на Питоне:

In [4]:
# Функция перехода из комнаты в комнату
def go(room):
    def func(state):
        return dict(state, room=room)
    return func


# Структура игры. Комнаты и допустимые в них действия
game = {
    'room0': dict(
        left=go('room1'),
        up=go('room2'),
        right=go('room3')
    ),
    'room1': dict(
        up=go('room2'),
        right=go('room0')
    ),
    'room2': dict(
    ),
    'room3': dict(
        up=go('room4'),
        right=go('room5')
    ),
    'room4': dict(
        down=go('room3'),
        right=go('room5')
    ),
    'room5': dict(
        up=go('room4'),
        left=go('room3')
    )
}

# Стартовое состояние
START_STATE = dict(room='room0')


def is_goal_state(s):
    '''
    Проверить, является ли состояние целевым.
    На входе ожидается множество пар ключ-значение.
    '''
    return ('room', 'room2') in s


def get_current_room(state):
    '''
    Выдать комнату, в которой находится игрок.
    '''
    return state['room']

1. Реализовать функцию `make_model`, которая по структуре игры и стартовому состоянию строит граф всех возможных состояний.
1. Реализовать функцию `find_dead_ends`, которая выдает список тупиковых узлов графа. Вспомните тупиковые ситуации из известных вам компьютерных игр, где, иной раз, для дальнейшего прохождения нужно прибегать к старому сохранению или вводу специальных системных команд, и все потому, что был потерян важный для дальнейшего прохождения предмет или не совершено требуемое действие.
1. Добавьте в одну из комнат room3-room5 рычаг, нажатие на который делает односторонний переход из room0 в room3 двусторонним.
1. (дополнительно) Придумайте минимальную игровую ситуацию, при которой будет наблюдаться комбинаторный взрыв для графа состояний.

Простой вариант функции печати графа в формате GraphViz:

In [2]:
def print_dot(graph, start_key):
    dead_ends = [] # find_dead_ends(graph) TODO
    print('digraph {')
    graph_keys = list(graph.keys())
    for x in graph:
        n = graph_keys.index(x)
        if x == start_key:
            print(f'n{n} [style="filled",fillcolor="dodgerblue",shape="circle"]')
        elif is_goal_state(x):
            print(f'n{n} [style="filled",fillcolor="green",shape="circle"]')
        elif x in dead_ends:
            print(f'n{n} [style="filled",fillcolor="red",shape="circle"]')
        else:
            print(f'n{n} [shape="circle"]')
    for x in graph:
        n1 = graph_keys.index(x)
        for y in graph[x]:
            n2 = graph_keys.index(y)
            print(f'n{n1} -> n{n2}')
    print('}')

Вот как выглядит вывод функции print_dot:
    
![](images/game.svg)

Вывод для пункта 4 с рычагом:
    
![](images/puzzlegraph2.svg)

**Задача 8**

Есть простая игра The Teeny Tiny Mansion (TTTM), описание которой приведено по [ссылке](http://svn.clifford.at/handicraft/2017/tttm/README). В этой игре имеется 2 персонажа (Алиса и Боб), которыми можно поочередно управлять, 4 комнаты, 3 двери и 3 ключа. Персонажи могут брать ключи и передавать их друг другу. Целью является привести Алису в красную комнату, а Боба — в голубую комнату.

В качестве стартового состояния используйте:

```Python

START_STATE = dict(
    player='alice',
    alice_room='west room',
    bob_room='east room',
    red_key='east room',
    blue_key='west room',
    green_key='east room'
)
```

1. Реализовать все пункты предыдущей задачи для TTTM.
1. Реализовать функцию подсчета кратчайшего числа шагов, за которые игру можно успешно завершить.
1. Провести верификацию для общего случая: Алиса, Боб и 3 ключа случайно располагаются в западной и восточной комнатах.

Пример вывода графа состояний:

![](images/tttm.svg)